In [13]:
import mlflow
import mlflow.keras
from mlflow.models import infer_signature
import numpy as np
from nightingale.model.classifier_head import ClassifierHead

# mlflow.tensorflow.autolog(
#     log_models=True,
#     log_input_examples=True,
#     log_model_signatures=True,
#     log_every_n_steps=1,
# )

model = ClassifierHead()
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")

input_data = np.random.random((128, 1024))
labels = np.random.randint(0, 2, size=(128,))

model.fit(
    x=input_data,
    y=labels,
    epochs=1
)

# Get predictions
sample_output = model.predict(input_data)
print(model.summary())

# Infer signature from data
signature = infer_signature(input_data, sample_output)

with mlflow.start_run() as run:
   model_info = mlflow.keras.log_model(model, name = "model", signature=signature, pip_requirements=['keras==3.10.0'])



4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 1.2134 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 


Model: "classifier_head_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_18 (Dense)                │ (32, 512)              │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (32, 3)                │         1,539 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,579,019 (6.02 MB)

 Trainable params: 526,339 (2.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,052,680 (4.02 MB)

None


In [14]:
loaded_model = mlflow.keras.load_model(model_info.model_uri)
print("Model uri: ", model_info.model_uri)
print(loaded_model.summary())

Model uri:  models:/m-e8ab8cbae5e045d0acf50a184ccc88c2


Model: "classifier_head_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_20 (Dense)                │ (32, 512)              │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (32, 3)                │         1,539 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,579,019 (6.02 MB)

 Trainable params: 526,339 (2.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,052,680 (4.02 MB)

None


In [15]:
# Test the loaded model produces the same output for the same input as the model.
test_input = np.random.random((128, 1024))
np.testing.assert_allclose(
    model.predict(test_input),
    loaded_model.predict(test_input),
)

1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
